# Configure Check Conditions

Conditions are logic which runs on the check results in order to rate it as passing value or not passing.

## Edit Conditions

Deepchecks provides different kind of default suites, which comes with built-in conditions. You may want to remove a condition in case it isn't needed for you, or you want to change the parameters (since conditions functions are immutable).

To remove condition, start with printing the Suite and identifing the Check's ID, and the Condition's ID:

In [1]:
from deepchecks.suites import overfit_suite

suite = overfit_suite()
suite

Overfit Suite: [
	0: TrainTestDifferenceOverfit
		Conditions:
			0: Train-Test metrics degradation ratio is not greater than 0.1
	1: BoostingOverfit(num_steps=20)
		Conditions:
			0: Test score decline is not greater than 5.00%
]

After we found the IDs we can remove the Condition:

In [2]:
# Access check by id
check = suite[1]
# Remove condition by id
check.remove_condition(0)

suite

Overfit Suite: [
	0: TrainTestDifferenceOverfit
		Conditions:
			0: Train-Test metrics degradation ratio is not greater than 0.1
	1: BoostingOverfit(num_steps=20)
]

Now if we want we can also re-add the Condition using the built-in methods on the check, with a different parameter. 

In [3]:
# Re-add the condition with new parameter
check.add_condition_test_score_percent_decline_not_greater_than(0.2)

suite

Overfit Suite: [
	0: TrainTestDifferenceOverfit
		Conditions:
			0: Train-Test metrics degradation ratio is not greater than 0.1
	1: BoostingOverfit(num_steps=20)
		Conditions:
			1: Test score decline is not greater than 20.00%
]

## Add a Custom Condition

In order to write conditions we first have to know what value a given check produces.  
Let's look at the check `DatasetsSizeComparison` and see it's return value in order to write condition for it.

In [4]:
from deepchecks.checks import DatasetsSizeComparison
from deepchecks import Dataset
import pandas as pd

# We'll use dummy data for the purpose of this demonstration
train_dataset = Dataset(pd.DataFrame(data={'x': [1,2,3,4,5,6,7,8,9]}))
test_dataset = Dataset(pd.DataFrame(data={'x': [1,2,3]}))

result = DatasetsSizeComparison().run(train_dataset, test_dataset)
result.value

{'Train': 9, 'Test': 3}

Now we know the return value looks like. let's add a new condition that validates the ratio between train and test sizes is inside a given range.
To create condition we need to use the `add_condition` method of the check which accepts condition name and a function. this function receives the value of the `CheckResult` that we saw above and should return a boolean or `ConditionResult`

*Note: When implementing a condition in a custom check, you may want to add a method `add_condition_x()` to allow any consumer of your check to apply the condition (possibly with given parameters). For examples look at implemented Checks' source code*

In [5]:
from deepchecks import ConditionResult

# Our parameters for the condition
low_threshold = 0.4
high_threshold = 0.6

# Create the condition function
def custom_condition(value: dict): 
    ratio = value['Test'] / value['Train']
    if low_threshold <= ratio <= high_threshold:
        return ConditionResult(True)
    else:
        # Note: if you doesn't care about the extra info, you can return directly a boolean
        return ConditionResult(False, f'Test-Train ratio is {ratio:.2}')

# Create the condition name
condition_name = f'Test-Train ratio is between {low_threshold} to {high_threshold}'

# Create check instance with the condition 
check = DatasetsSizeComparison().add_condition(condition_name, custom_condition)

Now we will use Suite to demonstrate the action of the condition, since the suite runs the condition for us automaticlly and print it in a stylish way

In [6]:
from deepchecks import Suite

# Using suite to run check & condition
suite = Suite('Suite for Condition',
    check
)

suite.run(train_dataset, test_dataset)

Suite for Condition:   0%|          | 0/1 [00:00<?, ? Check/s]

Suite for Condition 
 The suite is composed of various checks such as: Datasets Size Comparison, etc... 
 Each check may contain conditions (which results in 
 ✓ /
 ✖ /
 ! 
 ), as well as other outputs such as plots or tables. 
 Suites, checks and conditions can all be modified (see tutorial [link]). 
 Conditions Summary

Status,Check,Condition,More Info
✖,Datasets Size Comparison,Test-Train ratio is between 0.4 to 0.6,Test-Train ratio is 0.33


Additional Outputs

Datasets Size Comparison

Verify test dataset size comparing it to the train dataset size.

,Train,Test
Size,9,3
